In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [11]:
from tqdm import tqdm
from collections import defaultdict
import random
import json

In [3]:
import malaya

In [4]:
tokenizer = malaya.preprocessing.TOKENIZER().tokenize
sastrawi = malaya.stem.sastrawi()

In [9]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/dictionary/synonym/synonym0.json
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/dictionary/synonym/synonym1.json
files = ['synonym0.json', 'synonym1.json']
synonyms = defaultdict(list)

In [12]:
for file in files:
    with open(file) as fopen:
        data = json.load(fopen)
    for i in data:
        if not len(i[1]):
            continue
        synonyms[i[0]].extend(i[1])
        for r in i[1]:
            synonyms[r].append(i[0])
            
for k, v in synonyms.items():
    synonyms[k] = list(set(v))

In [ ]:
def replace(string, threshold, synonym):
    for no, word in enumerate(string):
        if word in synonym_dict and random.random() > threshold:
            w = random.choice(synonym[word])
            string[no] = w
    return string

In [16]:
import copy
import re
from malaya.text.regex import _expressions

def reset_t(tokens):
    t = []
    for i in range(len(tokens)):
        t.append([tokens[i], 2])
    return t

def augment_17_0(t, row, threshold = 0.5):
    text, tokens, tokens_lower = row
    for i in range(len(tokens)):
        if tokens_lower[i] in synonyms and random.random() > threshold:
            w = random.choice(synonyms[tokens_lower[i]])
            t[i][0] = w
            t[i][1] = 17

In [19]:
results = []
for text in tqdm(['Cerpen itu telah saya karang.', 
                  'Latihan itu mesti kau buat.',
                  'Kereta itu saya beli daripada Ali.',
                  'Dia memberi saya sebatang pena.']):
    tokens = tokenizer(text)
    t = reset_t(tokens)
    t_ = copy.deepcopy(t)
    tokens_lower = tokenizer(text.lower())
    r = (t, tokens, tokens_lower)
    augment_17_0(t_, r)
    results.append((t, t_))

100%|██████████| 4/4 [00:00<00:00, 3877.33it/s]


In [20]:
results

[([['Cerpen', 2],
   ['itu', 2],
   ['telah', 2],
   ['saya', 2],
   ['karang', 2],
   ['.', 2]],
  [['Cerpen', 2],
   ['itu', 2],
   ['pernah', 17],
   ['saya', 2],
   ['karang', 2],
   ['.', 2]]),
 ([['Latihan', 2],
   ['itu', 2],
   ['mesti', 2],
   ['kau', 2],
   ['buat', 2],
   ['.', 2]],
  [['senaman', 17],
   ['itu', 2],
   ['mesti', 2],
   ['kau', 2],
   ['mengatur', 17],
   ['.', 2]]),
 ([['Kereta', 2],
   ['itu', 2],
   ['saya', 2],
   ['beli', 2],
   ['daripada', 2],
   ['Ali', 2],
   ['.', 2]],
  [['Kereta', 2],
   ['itu', 2],
   ['saya', 2],
   ['memperoleh', 17],
   ['daripada', 2],
   ['Ali', 2],
   ['.', 2]]),
 ([['Dia', 2],
   ['memberi', 2],
   ['saya', 2],
   ['sebatang', 2],
   ['pena', 2],
   ['.', 2]],
  [['Dia', 2],
   ['menampung', 17],
   ['saya', 2],
   ['sebatang', 2],
   ['pena', 2],
   ['.', 2]])]